<a href="https://colab.research.google.com/github/alayacruz/task1/blob/main/Copy_of_recommender_systems_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [ ]:
df = pd.read_csv('/content/u.data', sep='\t', header=None, names=['user_id', 'movie_id', 'rating','timestamp'])
df.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [ ]:
column_names = [ 'movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western' ]
# Read the file
df_items = pd.read_csv( '/content/u.item', sep='|', header=None, names=column_names, encoding='ISO-8859-1' )
df_items=df_items.drop(columns=['video_release_date','IMDb_URL'])
df_items.head()

,movie_id,movie_title,release_date,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
df_genre=pd.read_csv('/content/u.genre', sep='|', header=None)
df_genre.head()

df_user= pd.read_csv('/content/u.user', sep='|', header=None, names=['user_id','age','gender','occupation','zip code'])
df_user.head()

,user_id,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [ ]:
n_rating=len(df)
n_users=len(df['user_id'].unique())
n_movies=len(df['movie_id'].unique())



In [ ]:
user_freq=df[['user_id','movie_id']].groupby('user_id').count().reset_index()
user_freq.columns=['user_id','n_ratings'] #assigning new names to the columns
print(user_freq.head())

   user_id  n_ratings
0        1        272
1        2         62
2        3         54
3        4         24
4        5        175


In [ ]:
mean_ratings=df.groupby('movie_id')[['rating']].mean()
lowest_rated=mean_ratings['rating'].idxmin() #find index(movie id ) corresponding to min value of the rating column
df_items.loc[df_items['movie_id']==lowest_rated] #displays row whose movie id matches the lowest rated movie id from movies dataframe

highest_rated=mean_ratings['rating'].idxmax()
df_items.loc[df_items['movie_id']==highest_rated]

df[df['movie_id']==highest_rated]
df[df['movie_id']==lowest_rated]

movie_stats=df.groupby('movie_id')[['rating']].agg(['count','mean'])

movie_stats.columns=movie_stats.columns.droplevel()


In [ ]:
from scipy.sparse import csr_matrix

def create_matrix(df):
  N=len(df['user_id'].unique())#it extracts all the unique values and returns it as an ndarray
  M=len(df['movie_id'].unique())


#to convert the user ids into numerical representation of integer values
  user_mapper=dict(zip(np.unique(df['user_id']), list(range(N))))
  movie_mapper=dict(zip(np.unique(df['movie_id']),list(range(M))))
   #np.unique() unique function of numpy that returns an ndarray of unique values of user id
#list(range(N)) creates a list of integer values
#The zip function combines the two sequences unique user ids and list of integers into iters of tuples
#finally dict converts the tuples into a dictionary with user ids as the keys

  user_inv_mapper=dict(zip(list(range(N)), np.unique(df["user_id"])))
  movie_inv_mapper=dict(zip(list(range(M)), np.unique(df['movie_id'])))

  user_index=[user_mapper[i] for i in df['user_id']]
  movie_index= [movie_mapper[i] for i in df['movie_id']]

  X=csr_matrix((df["rating"],(movie_index,user_index)), shape=(M,N))
  return X,user_mapper,movie_mapper,user_inv_mapper,movie_inv_mapper

X,user_mapper,movie_mapper,user_inv_mapper,movie_inv_mapper=create_matrix(df)
print(X)

  (0, 0)	5
  (0, 1)	4
  (0, 4)	4
  (0, 5)	4
  (0, 9)	4
  (0, 12)	3
  (0, 14)	1
  (0, 15)	5
  (0, 16)	4
  (0, 17)	5
  (0, 19)	3
  (0, 20)	5
  (0, 22)	5
  (0, 24)	5
  (0, 25)	3
  (0, 37)	5
  (0, 40)	4
  (0, 41)	5
  (0, 42)	5
  (0, 43)	4
  (0, 44)	5
  (0, 48)	2
  (0, 53)	4
  (0, 55)	4
  (0, 56)	5
  :	:
  (1661, 781)	4
  (1662, 781)	2
  (1663, 781)	4
  (1663, 838)	1
  (1663, 869)	4
  (1663, 879)	4
  (1664, 781)	2
  (1665, 781)	2
  (1666, 781)	3
  (1667, 781)	3
  (1668, 781)	2
  (1669, 781)	3
  (1670, 786)	1
  (1671, 827)	2
  (1671, 895)	2
  (1672, 834)	3
  (1673, 839)	4
  (1674, 850)	3
  (1675, 850)	2
  (1676, 853)	3
  (1677, 862)	1
  (1678, 862)	3
  (1679, 862)	2
  (1680, 895)	3
  (1681, 915)	3


In [ ]:
from sklearn.neighbors import NearestNeighbors

def find_similar_movies(movie_id,X,k,metric='cosine',show_distance=False):
  neighbour_ids=[]
  movie_ind=movie_mapper[movie_id]
  movie_vec=X[movie_ind]
  k+=1
  knn=NearestNeighbors(n_neighbors=k,algorithm="brute",metric=metric)
  knn.fit(X)
  movie_vec=movie_vec.reshape(1,-1)
  neighbor=knn.kneighbors(movie_vec,return_distance=show_distance)
  for i in range(0,k):
    n=neighbor.item(i)
    neighbour_ids.append(movie_inv_mapper[n])
  neighbour_ids.pop(0)
  return neighbour_ids

movie_titles=dict(zip(df_items['movie_id'],df_items['movie_title']))

movie_id=1

similar_movies=find_similar_movies(movie_id,X,k=10)
movie_title=movie_titles[movie_id]



print(f"since you watched {movie_title}")
for i in similar_movies:
  print(movie_titles[i])





since you watched Toy Story (1995)
Star Wars (1977)
Return of the Jedi (1983)
Independence Day (ID4) (1996)
Rock, The (1996)
Mission: Impossible (1996)
Willy Wonka and the Chocolate Factory (1971)
Star Trek: First Contact (1996)
Fargo (1996)
Jerry Maguire (1996)
Raiders of the Lost Ark (1981)


In [ ]:

genre_cols = df_items.columns[4:]

# Create a new DataFrame with movie IDs and genres
movies_genres = df_items[['movie_id'] + list(genre_cols)]
# One-hot encoding
movies_encoded = pd.get_dummies(movies_genres, columns=genre_cols)
print(movies_encoded.head())

   movie_id  Action_0  Action_1  Adventure_0  Adventure_1  Animation_0  \
0         1      True     False         True        False        False   
1         2     False      True        False         True         True   
2         3      True     False         True        False         True   
3         4     False      True         True        False         True   
4         5      True     False         True        False         True   

   Animation_1  Children's_0  Children's_1  Comedy_0  ...  Romance_0  \
0         True         False          True     False  ...       True   
1        False          True         False      True  ...       True   
2        False          True         False      True  ...       True   
3        False          True         False     False  ...       True   
4        False          True         False      True  ...       True   

   Romance_1  Sci-Fi_0  Sci-Fi_1  Thriller_0  Thriller_1  War_0  War_1  \
0      False      True     False        True    

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(movies_encoded.drop('movie_id', axis=1), movies_encoded.drop('movie_id', axis=1))

In [ ]:
def genre_based_recommend(movie_id, cosine_sim, movies_encoded, k=10):
    # Get the index of the movie
    movie_index = movies_encoded[movies_encoded['movie_id'] == movie_id].index[0]

    # Get the similarity scores for the movie
    similarity_scores = cosine_sim[movie_index]

    # Get the indices of the most similar movies
    similar_movie_indices = similarity_scores.argsort()[::-1][1:k+1]  # Exclude the movie itself

    # Get the movie IDs of the most similar movies
    similar_movie_ids = movies_encoded.iloc[similar_movie_indices]['movie_id'].tolist()

    return similar_movie_ids

recommendations = genre_based_recommend(movie_id=1, cosine_sim=cosine_sim, movies_encoded=movies_encoded)
movie_title=movie_titles[1]



print(f"since you watched {movie_title}")
for i in recommendations:
  print(movie_titles[i])

since you watched Toy Story (1995)
Aladdin and the King of Thieves (1996)
Wrong Trousers, The (1993)
Goofy Movie, A (1995)
Mouse Hunt (1997)
Home Alone (1990)
Aladdin (1992)
Aristocats, The (1970)
Little Big League (1994)
Matilda (1996)
Air Bud (1997)


In [ ]:
def hybrid_recommend(movie_id, X, k_item=10, k_content=10, content_weight=0.5):

    # Get item-based recommendations
    collab_recommendations = find_similar_movies(movie_id, X, k=k_item)

    # Get content-based recommendations
    content_recommendations = genre_based_recommend(movie_id, cosine_sim, movies_encoded, k=k_content)

    # Combine recommendations using a weighted ranking
    all_recommendations = collab_recommendations + content_recommendations
    unique_recommendations = list(dict.fromkeys(all_recommendations)) # Remove duplicates

    # Score recommendations based on their presence in both lists and weight
    scores = {}
    for movie in unique_recommendations:
        score = 0
        if movie in collab_recommendations:
            score += 1
        if movie in content_recommendations:
            score += content_weight  # Adjust weight as needed
        scores[movie] = score

    # Sort recommendations by score and return the top k
    sorted_recommendations = sorted(scores.items(), key=lambda item: item[1], reverse=True)
    top_recommendations = [movie_id for movie_id, score in sorted_recommendations][:k_item]  # Take top k_item recommendations

    return top_recommendations
movie_id=1
hybrid_recs=hybrid_recommend(movie_id,X,content_weight=0.7)
print(f"since you watched {movie_titles[movie_id]}")
for i in hybrid_recs:
  print(movie_titles[i])

since you watched Toy Story (1995)
Star Wars (1977)
Return of the Jedi (1983)
Independence Day (ID4) (1996)
Rock, The (1996)
Mission: Impossible (1996)
Willy Wonka and the Chocolate Factory (1971)
Star Trek: First Contact (1996)
Fargo (1996)
Jerry Maguire (1996)
Raiders of the Lost Ark (1981)


In [ ]:
!pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.4 MB/s eta 0:00:00


In [ ]:
import streamlit as st
movie_list=list(movie_titles.values())

selected_movie=st.selectbox("select a movie you have watched before", movie_list)
movie_id=list(movie_titles.keys())[list(movie_titles.values()).index(selected_movie)]

st.title("movies recommendation system")

if st.button("Get Recommendations"):
    recommendations = hybrid_recommend(movie_id, X)  # Your recommendation function
    st.write(f"Since you watched {selected_movie}:")
    for i in recommendations:
        st.write(movie_titles[i])



2025-02-02 16:47:11.904 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-02 16:47:11.905 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-02 16:47:11.913 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-02 16:47:11.918 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-02 16:47:11.920 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-02-02 16:47:11.924 WARNING streamlit.runtime.scriptrunner_utils.script_run_c

In [ ]:
!streamlit run recommender_systems.py

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: recommender_systems.py
